In [1]:
import scipy.misc
import random

train_batch_pointer=0

validation_batch_pointer=0

from itertools import islice

images=[]

angles=[]

from scipy import pi

txt=open("driving_dataset/driving_dataset/data.txt")
num_lines=sum(1 for line in open("driving_dataset/driving_dataset/data.txt"))
lines_slice=islice(txt,num_lines)

for line in lines_slice:
	image,angle=line.strip().split()
	images.append("driving_dataset/driving_dataset/"+image)
	angle=float(angle)*scipy.pi/180
	angles.append(angle)

#time based split
split_ratio=0.8
split_up_to=int(num_lines*split_ratio)
print(split_up_to)
train_images=images[:split_up_to]
train_angles=angles[:split_up_to]

validation_images=images[split_up_to:]
validation_angles=angles[split_up_to:]

num_train_images=len(train_images)
num_validation_images=len(validation_images)
print(num_train_images,num_validation_images)

36324
36324 9082


In [2]:
train_ang=[]
import datetime  as dt
start=dt.datetime.now()
for i in range(0,36324):
    train_ang.append([train_angles[i]])
    
print("time took:",dt.datetime.now()-start)



val_im=[]

import datetime  as dt
start=dt.datetime.now()
for i in range(0,9082):
    val_im.append(scipy.misc.imresize(scipy.misc.imread(validation_images[ i ])[-150:], [66, 200]) / 255.0)
print("time took:",dt.datetime.now()-start)



val_ang=[]
import datetime  as dt
start=dt.datetime.now()
for i in range(0,9082):
    val_ang.append([validation_angles[i]])
    
print("time took:",dt.datetime.now()-start)


train_im=[]
import datetime  as dt
start=dt.datetime.now()
for i in range(0,36324):
    train_im.append(scipy.misc.imresize(scipy.misc.imread(train_images[ i ])[-150:], [66, 200]) / 255.0)
    
print("time took:",dt.datetime.now()-start)


start = dt.datetime.now()

import numpy as np
train_im=np.array(train_im)
val_im=np.array(val_im)
val_ang=np.array(val_ang)
train_ang=np.array(train_ang)

print("time took::",dt.datetime.now()-start)

time took: 0:00:00.009006


C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  app.launch_new_instance()


time took: 0:00:27.085960
time took: 0:00:00.002992


C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


time took: 0:04:07.663471
time took:: 0:02:05.606938


In [3]:
from keras.models import Sequential 
from keras.layers import Dense,Flatten,Dropout,Conv2D
from keras import backend as k
from keras.initializers import TruncatedNormal,Constant
model = Sequential()

model.add(Conv2D(24, (5,5), strides=(2,2), activation="relu",input_shape = (66,200,3) ))
model.add(Conv2D(36, (5,5), strides=(2,2), activation="relu" ))
model.add(Conv2D(48, (5,5), strides=(2,2), activation="relu" ))
model.add(Conv2D(64, (3,3), strides=(2,2), activation="relu" ))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(100))
model.add(Dropout(0.3))
model.add(Dense(50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 10, 64)         27712     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)  

In [4]:
start=dt.datetime.now()
model.compile(loss='mse', optimizer='Adam')
import time
import keras
class delay(keras.callbacks.Callback):
    def __int__(self,delay_time=600):
        sel.delay_time = 600
        
    def on_epoch_begin(self,epoch,logs={}):
        if(epoch+1)%100==0:
            print("cooling down to decrease CPU tempereture")
            time.sleep(300)#600 sec
results=model.fit(train_im,train_ang,epochs=1000,batch_size=512,validation_data=(val_im,val_ang),verbose=1,callbacks=[delay()])
print("time took",dt.datetime.now()-start)

Instructions for updating:
Use tf.cast instead.
Train on 36324 samples, validate on 9082 samples
Epoch 1/1000
36324/36324 [==============================] - 82s 2ms/step - loss: 0.2644 - val_loss: 0.2032
Epoch 2/1000
36324/36324 [==============================] - 54s 1ms/step - loss: 0.2003 - val_loss: 0.2378
Epoch 3/1000
36324/36324 [==============================] - 45s 1ms/step - loss: 0.1396 - val_loss: 0.2635
Epoch 4/1000
36324/36324 [==============================] - 44s 1ms/step - loss: 0.0980 - val_loss: 0.3144
Epoch 5/1000
36324/36324 [==============================] - 42s 1ms/step - loss: 0.0999 - val_loss: 0.2673
Epoch 6/1000
36324/36324 [==============================] - 41s 1ms/step - loss: 0.0685 - val_loss: 0.2447
Epoch 7/1000
36324/36324 [==============================] - 43s 1ms/step - loss: 0.0644 - val_loss: 0.2194
Epoch 8/1000
36324/36324 [==============================] - 44s 1ms/step - loss: 0.0479 - val_loss: 0.2061
Epoch 9/1000
36324/36324 [=====================

Epoch 74/1000
36324/36324 [==============================] - 39s 1ms/step - loss: 0.0159 - val_loss: 0.2215
Epoch 75/1000
36324/36324 [==============================] - 73s 2ms/step - loss: 0.0154 - val_loss: 0.1859
Epoch 76/1000
36324/36324 [==============================] - 99s 3ms/step - loss: 0.0151 - val_loss: 0.1730
Epoch 77/1000
36324/36324 [==============================] - 80s 2ms/step - loss: 0.0150 - val_loss: 0.1961
Epoch 78/1000
36324/36324 [==============================] - 80s 2ms/step - loss: 0.0144 - val_loss: 0.1761
Epoch 79/1000
36324/36324 [==============================] - 84s 2ms/step - loss: 0.0151 - val_loss: 0.1686
Epoch 80/1000
36324/36324 [==============================] - 83s 2ms/step - loss: 0.0169 - val_loss: 0.2059
Epoch 81/1000
36324/36324 [==============================] - 77s 2ms/step - loss: 0.0160 - val_loss: 0.2109
Epoch 82/1000
36324/36324 [==============================] - 83s 2ms/step - loss: 0.0159 - val_loss: 0.1910
Epoch 83/1000
36324/36324 [=

36324/36324 [==============================] - 34s 949us/step - loss: 0.0108 - val_loss: 0.1752
Epoch 223/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0127 - val_loss: 0.1672
Epoch 224/1000
36324/36324 [==============================] - 43s 1ms/step - loss: 0.0134 - val_loss: 0.1784
Epoch 225/1000
36324/36324 [==============================] - 46s 1ms/step - loss: 0.0115 - val_loss: 0.1843
Epoch 226/1000
36324/36324 [==============================] - 34s 927us/step - loss: 0.0112 - val_loss: 0.1718
Epoch 227/1000
36324/36324 [==============================] - 36s 995us/step - loss: 0.0097 - val_loss: 0.1785
Epoch 228/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0122 - val_loss: 0.1684
Epoch 229/1000
36324/36324 [==============================] - 36s 1ms/step - loss: 0.0101 - val_loss: 0.1724
Epoch 230/1000
36324/36324 [==============================] - 36s 999us/step - loss: 0.0102 - val_loss: 0.1805
Epoch 231/1000
36324/36324

Epoch 297/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0118 - val_loss: 0.1948
Epoch 298/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0121 - val_loss: 0.1727
Epoch 299/1000
36324/36324 [==============================] - 36s 996us/step - loss: 0.0095 - val_loss: 0.1813
Epoch 300/1000
cooling down to decrease CPU tempereture
36324/36324 [==============================] - 378s 10ms/step - loss: 0.0106 - val_loss: 0.1737
Epoch 301/1000
36324/36324 [==============================] - 43s 1ms/step - loss: 0.0116 - val_loss: 0.1750
Epoch 302/1000
36324/36324 [==============================] - 42s 1ms/step - loss: 0.0102 - val_loss: 0.1726
Epoch 303/1000
36324/36324 [==============================] - 40s 1ms/step - loss: 0.0093 - val_loss: 0.1755
Epoch 304/1000
36324/36324 [==============================] - 38s 1ms/step - loss: 0.0104 - val_loss: 0.1777
Epoch 305/1000
36324/36324 [==============================] - 39s 1ms/step - loss: 

36324/36324 [==============================] - 50s 1ms/step - loss: 0.0085 - val_loss: 0.1634
Epoch 446/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0093 - val_loss: 0.1700
Epoch 447/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0083 - val_loss: 0.1736
Epoch 448/1000
36324/36324 [==============================] - 56s 2ms/step - loss: 0.0100 - val_loss: 0.1675
Epoch 449/1000
36324/36324 [==============================] - 35s 977us/step - loss: 0.0100 - val_loss: 0.1772
Epoch 450/1000
36324/36324 [==============================] - 34s 946us/step - loss: 0.0090 - val_loss: 0.1711
Epoch 451/1000
36324/36324 [==============================] - 34s 946us/step - loss: 0.0085 - val_loss: 0.1818
Epoch 452/1000
36324/36324 [==============================] - 42s 1ms/step - loss: 0.0089 - val_loss: 0.1629
Epoch 453/1000
36324/36324 [==============================] - 43s 1ms/step - loss: 0.0096 - val_loss: 0.1751
Epoch 454/1000
36324/36324 [

Epoch 594/1000
36324/36324 [==============================] - 56s 2ms/step - loss: 0.0081 - val_loss: 0.1769
Epoch 595/1000
36324/36324 [==============================] - 80s 2ms/step - loss: 0.0087 - val_loss: 0.1702
Epoch 596/1000
36324/36324 [==============================] - 105s 3ms/step - loss: 0.0092 - val_loss: 0.1773
Epoch 597/1000
36324/36324 [==============================] - 87s 2ms/step - loss: 0.0096 - val_loss: 0.1852
Epoch 598/1000
36324/36324 [==============================] - 79s 2ms/step - loss: 0.0092 - val_loss: 0.1822
Epoch 599/1000
36324/36324 [==============================] - 86s 2ms/step - loss: 0.0085 - val_loss: 0.1689
Epoch 600/1000
cooling down to decrease CPU tempereture
36324/36324 [==============================] - 415s 11ms/step - loss: 0.0084 - val_loss: 0.1707
Epoch 601/1000
36324/36324 [==============================] - 85s 2ms/step - loss: 0.0094 - val_loss: 0.1695
Epoch 602/1000
36324/36324 [==============================] - 78s 2ms/step - loss: 0

36324/36324 [==============================] - 36s 995us/step - loss: 0.0088 - val_loss: 0.1671
Epoch 743/1000
36324/36324 [==============================] - 38s 1ms/step - loss: 0.0079 - val_loss: 0.1789
Epoch 744/1000
36324/36324 [==============================] - 41s 1ms/step - loss: 0.0085 - val_loss: 0.1778
Epoch 745/1000
36324/36324 [==============================] - 40s 1ms/step - loss: 0.0089 - val_loss: 0.1765
Epoch 746/1000
36324/36324 [==============================] - 39s 1ms/step - loss: 0.0086 - val_loss: 0.1744
Epoch 747/1000
36324/36324 [==============================] - 49s 1ms/step - loss: 0.0117 - val_loss: 0.1702
Epoch 748/1000
36324/36324 [==============================] - 38s 1ms/step - loss: 0.0089 - val_loss: 0.1823
Epoch 749/1000
36324/36324 [==============================] - 37s 1ms/step - loss: 0.0084 - val_loss: 0.1665
Epoch 750/1000
36324/36324 [==============================] - 36s 978us/step - loss: 0.0085 - val_loss: 0.1722
Epoch 751/1000
36324/36324 [==

Epoch 891/1000
36324/36324 [==============================] - 47s 1ms/step - loss: 0.0089 - val_loss: 0.1756
Epoch 892/1000
36324/36324 [==============================] - 38s 1ms/step - loss: 0.0091 - val_loss: 0.1884
Epoch 893/1000
36324/36324 [==============================] - 39s 1ms/step - loss: 0.0082 - val_loss: 0.1780
Epoch 894/1000
36324/36324 [==============================] - 39s 1ms/step - loss: 0.0078 - val_loss: 0.1712
Epoch 895/1000
36324/36324 [==============================] - 36s 999us/step - loss: 0.0086 - val_loss: 0.1793
Epoch 896/1000
36324/36324 [==============================] - 40s 1ms/step - loss: 0.0086 - val_loss: 0.1697
Epoch 897/1000
36324/36324 [==============================] - 53s 1ms/step - loss: 0.0087 - val_loss: 0.1683
Epoch 898/1000
36324/36324 [==============================] - 106s 3ms/step - loss: 0.0091 - val_loss: 0.1825
Epoch 899/1000
36324/36324 [==============================] - 85s 2ms/step - loss: 0.0076 - val_loss: 0.1690
Epoch 900/1000
c

36324/36324 [==============================] - 34s 928us/step - loss: 0.0085 - val_loss: 0.1615
Epoch 966/1000
36324/36324 [==============================] - 33s 898us/step - loss: 0.0087 - val_loss: 0.1793
Epoch 967/1000
36324/36324 [==============================] - 40s 1ms/step - loss: 0.0084 - val_loss: 0.1780
Epoch 968/1000
36324/36324 [==============================] - 35s 957us/step - loss: 0.0087 - val_loss: 0.1762
Epoch 969/1000
36324/36324 [==============================] - 33s 900us/step - loss: 0.0074 - val_loss: 0.1724
Epoch 970/1000
36324/36324 [==============================] - 38s 1ms/step - loss: 0.0078 - val_loss: 0.1790
Epoch 971/1000
36324/36324 [==============================] - 36s 994us/step - loss: 0.0087 - val_loss: 0.1673
Epoch 972/1000
36324/36324 [==============================] - 36s 987us/step - loss: 0.0087 - val_loss: 0.1847
Epoch 973/1000
36324/36324 [==============================] - 35s 961us/step - loss: 0.0078 - val_loss: 0.1652
Epoch 974/1000
36324

In [5]:
model.save("Best_model_One/mymodel_best_model.ckpt")

In [6]:
import pandas as pd
df=pd.DataFrame(results.history)

In [7]:
hist_csv_file = 'Best_model_One/history.csv'
with open(hist_csv_file, mode='w') as f:
    df.to_csv(f)

In [8]:
hist_json_file = 'Best_model_One/history.json' 
with open(hist_json_file, mode='w') as f:
    df.to_json(f)

In [9]:
export_excel = df.to_excel (r'Best_model_One/export_dataframe.xlsx', index = None, header=True)

In [11]:
scores = model.evaluate(train_im, train_ang, verbose=1)
print("loss on train data::")
print(scores)
scores = model.evaluate(val_im, val_ang, verbose=1)
print("loss on test data::")
scores

36324/36324 [==============================] - 77s 2ms/step
loss on train data::
0.0022747963839580675
9082/9082 [==============================] - 12s 1ms/step
loss on test data::


0.17691543988013494

In [12]:
import matplotlib.pyplot as plt




# Plot training & validation loss values
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

In [13]:
df

,val_loss,loss
0,0.203225,0.264360
1,0.237757,0.200286
2,0.263508,0.139577
3,0.314412,0.098040
4,0.267303,0.099887
5,0.244712,0.068521
6,0.219361,0.064429
7,0.206102,0.047933
8,0.221745,0.043400
9,0.252096,0.041663


In [14]:
import numpy as np
np.argmin(df["loss"])

C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  return getattr(obj, method)(*args, **kwds)


828

In [15]:
np.argmin(np.array(df["loss"]))

828

In [16]:
df[['loss']].idxmin() 

loss    828
dtype: int64

In [17]:
df[df['loss']==df['loss'][828]]

,val_loss,loss
828,0.169029,0.006902


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
val_loss    1000 non-null float64
loss        1000 non-null float64
dtypes: float64(2)
memory usage: 15.7 KB


In [19]:
df.describe()

,val_loss,loss
count,1000.000000,1000.000000
mean,0.175964,0.011505
std,0.011284,0.012496
min,0.155158,0.006902
25%,0.169665,0.008526
50%,0.174155,0.009399
75%,0.179036,0.011047
max,0.314412,0.264360


In [20]:
df["loss"].mean()

0.011504845455856644

In [21]:
df["val_loss"].mean()

0.17596382461970234

In [22]:
df[["val_loss"]].idxmin()

val_loss    487
dtype: int64

In [24]:
df[df["val_loss"]==df["val_loss"][487]]

,val_loss,loss
487,0.155158,0.009319


In [31]:
l = df["val_loss"]

In [33]:
l.min()

0.15515779092927595

In [34]:
df[df["val_loss"]==0.15515779092927595]

,val_loss,loss
487,0.155158,0.009319


In [35]:
df["loss"].min()

0.006901816320611217

In [4]:
from keras.models import load_model

In [6]:
model=load_model("Best_model_One/mymodel_best_model.ckpt")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [7]:
import cv2
img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0

i = 0
from subprocess import call


In [14]:
while(cv2.waitKey(20) != ord('q')):
    full_image = scipy.misc.imread("driving_dataset/driving_dataset/" + str(i) + ".jpg", mode="RGB")
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    degrees = model.predict(image[None,...])[0][0] * 180.0 / scipy.pi
    #call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
C:\Users\saireddyavs\Anaconda3\envs\gpuu\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  This is separate from the ipykernel package so we can avoid doing imports until


Predicted steering angle: -16.286138698504768 degrees
Predicted steering angle: -5.102702847775004 degrees
Predicted steering angle: -17.174162328349393 degrees
Predicted steering angle: -14.774153382317387 degrees
Predicted steering angle: -16.858943975511426 degrees
Predicted steering angle: -16.56294748254517 degrees
Predicted steering angle: -17.699567800063452 degrees
Predicted steering angle: -17.888769162714684 degrees
Predicted steering angle: -18.690225267471273 degrees
Predicted steering angle: -19.283510866704212 degrees
Predicted steering angle: -17.13113725922019 degrees
Predicted steering angle: -15.130025021095268 degrees
Predicted steering angle: -15.032645306551103 degrees
Predicted steering angle: -18.35076657326893 degrees
Predicted steering angle: -19.978788265718368 degrees
Predicted steering angle: -17.720264980795882 degrees
Predicted steering angle: -17.420112317719674 degrees
Predicted steering angle: -17.847624103154526 degrees
Predicted steering angle: -17.64

Predicted steering angle: -13.376693263038192 degrees
Predicted steering angle: -13.9641783241185 degrees
Predicted steering angle: -13.24150332879613 degrees
Predicted steering angle: -11.95007591996586 degrees
Predicted steering angle: -12.932256268840048 degrees
Predicted steering angle: -13.979563325223953 degrees
Predicted steering angle: -14.179153405602772 degrees
Predicted steering angle: -16.25429123395229 degrees
Predicted steering angle: -15.14266599370167 degrees
Predicted steering angle: -14.611512917801043 degrees
Predicted steering angle: -12.109186030454953 degrees
Predicted steering angle: -15.918471323030275 degrees
Predicted steering angle: -16.59045948452198 degrees
Predicted steering angle: -14.028561394782331 degrees
Predicted steering angle: -14.054608321182176 degrees
Predicted steering angle: -18.714817363577904 degrees
Predicted steering angle: -16.661664206619363 degrees
Predicted steering angle: -16.268276046277894 degrees
Predicted steering angle: -15.94761